# Proyecto Final


- Alfredo Osuna
- Raul Gonzalez

#### Librerias Necesarias

In [177]:
(require '[clojure.string :as s])
(use 'clojure.core.matrix)
(defn toInt[x] (Integer/parseInt x));Java
(defn toDouble[x] (Double/parseDouble x));Java

Execution error (FileNotFoundException) at user/eval4313 (REPL:2).
Could not locate clojure/core/matrix__init.class, clojure/core/matrix.clj or clojure/core/matrix.cljc on classpath.


class java.io.FileNotFoundException: 

Clase de Pelicula

In [15]:
(defrecord Movie [x0 imdb runtime rottenTomato]
    Object 
    (toString [this]
              (format "%s,%s,%s,%s" x0 imdb runtime rottenTomato)
              )
    )

user.Movie

Lectura de informacion

In [17]:
(def data (vec (rest (s/split-lines (slurp "Clean-Data.csv") ) ) ) )

#'user/data

Conversion de datos

In [18]:
(defn toMovie[ record ]
    (Movie. 
         (toInt(get record 0))  
         (toDouble (get record 1))
         (toInt (get record 2)) 
         (toDouble (get record 3))
     )
)

#'user/toMovie

In [19]:
(defn get-Movies-lazy [datos] 
    (lazy-seq (cons (toMovie (s/split (first datos) #","))(get-Movies-lazy (rest datos))) )
    )

#'user/get-Movies-lazy

In [20]:
(defn get-Vector-Valor [datos, valor]
    (lazy-seq (cons (valor (first datos))(get-Vector-Valor (rest datos) valor)))
    )

#'user/get-Vector-Valor

In [21]:
(def movies (take 20 (get-Movies-lazy data)))

#'user/movies

In [22]:
(def imdb (vec (take 20 (get-Vector-Valor movies :imdb))))

#'user/imdb

In [215]:
(def rottenTomato (vec (take 20 (get-Vector-Valor movies :rottenTomato))))

#'user/rottenTomato

In [10]:
(def runtime (vec (take 20 (get-Vector-Valor movies :runtime))))

#'user/runtime

In [24]:
(def x0 (vec (take 20 (get-Vector-Valor movies :x0))))

#'user/x0

In [178]:
(def  matrix_transpuesta [x0 imdb runtime])

#'user/matrix_transpuesta

In [179]:
 matrix_transpuesta

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] [5.9 5.0 8.2 5.3 7.0 6.5 5.9 6.6 5.1 5.5 6.4 6.6 7.7 7.8 5.0 5.6 5.6 5.4 7.4 8.0] [61 90 83 60 68 132 166 56 105 152 60 90 135 75 102 89 50 130 87 91]]

## Transpuesta Xt

In [180]:
(defn get-New-Col [matrix]
    (def newCol [(first (get matrix 0))(first (get matrix 1)) (first (get matrix 2))])
    newCol
    )

#'user/get-New-Col

In [181]:
(defn transpuesta [matrix]
    (loop [mResult (atom '()) mTemp matrix] 
         (if (= 0 (count (get mTemp 1)))
             mResult
             (do
                 (def m (get-New-Col mTemp))
                 (swap! mResult conj m)
                 (recur mResult [(vec (rest (get mTemp 0)))(vec (rest (get mTemp 1))) (vec (rest (get mTemp 2)))])
                 )
             )
        )
    )

#'user/transpuesta

In [182]:
(def matrixA (vec @(transpuesta matrixA)))

#'user/matrixA

## Multiplicador de matrices (Xt X)


In [216]:
 matrix_transpuesta

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] [5.9 5.0 8.2 5.3 7.0 6.5 5.9 6.6 5.1 5.5 6.4 6.6 7.7 7.8 5.0 5.6 5.6 5.4 7.4 8.0] [61 90 83 60 68 132 166 56 105 152 60 90 135 75 102 89 50 130 87 91]]

In [217]:
matrixA

[[1 8.0 91] [1 7.4 87] [1 5.4 130] [1 5.6 50] [1 5.6 89] [1 5.0 102] [1 7.8 75] [1 7.7 135] [1 6.6 90] [1 6.4 60] [1 5.5 152] [1 5.1 105] [1 6.6 56] [1 5.9 166] [1 6.5 132] [1 7.0 68] [1 5.3 60] [1 8.2 83] [1 5.0 90] [1 5.9 61]]

In [218]:
(defn get-Mult-Val-Xt-X [matrixA matrixB x]
     (loop [newValue 0 n 0]
       (if (= n (count matrixA))
            newValue
            (do
                (recur (+ (* (get matrixA n)(get (get matrixB n) x)) newValue) (inc n))    
            )
        )
    )
)

#'user/get-Mult-Val-Xt-X

In [219]:
(defn multiplicacion_x_xt [matrixA matrixTranspuesta]
    (loop [mResult (atom '())  n 0]
         (if (= n (count matrixTranspuesta))
             mResult
             (do
                 (def vectorTemp [(get-Mult-Val-Xt-X (get matrixTranspuesta n) matrixA 0) (get-Mult-Val-Xt-X (get matrixTranspuesta n) matrixA 1) (get-Mult-Val-Xt-X (get matrixTranspuesta n) matrixA 2)])                 
                 (swap! mResult conj vectorTemp)
                 (recur mResult (inc n))
                 
             )
        )
    )
)

#'user/multiplicacion_x_xt

In [220]:
(def matrix_multiplicacion_xt_x (vec (reverse @(multiplicacion_x_xt matrixA matrix_transpuesta))))

#'user/matrix_multiplicacion_xt_x

In [221]:
matrix_multiplicacion_xt_x

[[20 126.5 1882] [126.5 791.12 12082.199999999999] [1882 12082.199999999999 170534]]

## Inversa (Xt X)^-1

## Multiplicacion Xt Y

In [222]:
(defn get-Mult-Val-Xt-Y [matrixA matrixB]
     (loop [newValue 0 n 0]
       (if (= n (count matrixA))
            newValue
            (do
                (recur (+ (* (get matrixA n)(get matrixB n)) newValue) (inc n))    
            )
        )
    )
)

#'user/get-Mult-Val-Xt-Y

In [223]:
(defn matrix_multiplicacion_xt_y [matrixA matrixTranspuesta]
    (println matrixA)
    (def vectorTemp [(get-Mult-Val-Xt-Y (get matrixTranspuesta 0) matrixA)(get-Mult-Val-Xt-Y (get matrixTranspuesta 1) matrixA)(get-Mult-Val-Xt-Y (get matrixTranspuesta 2) matrixA)])                               
    (println vectorTemp)
    )

#'user/matrix_multiplicacion_xt_y

In [225]:
(def matrix_multiplicacion_xt_y (vec (reverse @(matrix_multiplicacion_xt_y rottenTomato matrix_transpuesta))))

[0.77 0.14 1.0 1.0 0.97 0.92 0.88 0.73 0.91 0.57 0.36 0.17 0.8 0.82 0.5 0.11 0.49 0.73 0.83 1.0]
[13.700000000000001 89.225 1308.92]


Execution error (NullPointerException) at nrepl.middleware.interruptible-eval/evaluate (interruptible_eval.clj:79).
null


class clojure.lang.Compiler$CompilerException: 